In [1]:
import os

In [2]:
%pwd

'/home/aradhyac/NephroScan-AI/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/aradhyac/NephroScan-AI'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
print(sys.executable)


/home/aradhyac/NephroScan-AI/venv/bin/python


In [7]:
from nephroscan_ai.constants import *
from nephroscan_ai.utils.common import read_yaml, create_directories
from nephroscan_ai.entity.config_entity import DataIngestionConfig


In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

        return data_ingestion_config


In [9]:
import os
import zipfile
import gdown
from nephroscan_ai import logger
from nephroscan_ai.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
import os
import zipfile
import gdown
from nephroscan_ai import logger
from nephroscan_ai.utils.common import get_size

In [21]:
import os
import zipfile
import gdown

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            
            # ✅ Show progress bar + resume download
            gdown.download(
                prefix + file_id,
                str(zip_download_dir),
                quiet=False,
                resume=True
            )

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(str(unzip_path), exist_ok=True)

        # ✅ Convert Path → str
        with zipfile.ZipFile(str(self.config.local_data_file), 'r') as zip_ref:
            zip_ref.extractall(str(unzip_path))


In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-19 12:12:14,027: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-19 12:12:14,032: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-19 12:12:14,036: INFO: common: created directory at: artifacts]
[2025-08-19 12:12:14,039: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-19 12:12:14,042: INFO: 2249439211: Downloading data from https://drive.google.com/file/d/1YYbpicJBzIF2QXHyPgQkoWrb1q_VQh6J/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
Resume: artifacts/data_ingestion/data.ziplhutl_ir.part
From (original): https://drive.google.com/uc?/export=download&id=1YYbpicJBzIF2QXHyPgQkoWrb1q_VQh6J
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1YYbpicJBzIF2QXHyPgQkoWrb1q_VQh6J&confirm=t&uuid=b3b2be85-a81c-4234-9a64-6752b3883d0f
To: /home/aradhyac/NephroScan-AI/artifacts/data_ingestion/data.zip


























































































































































































































100%|██████████| 1.63G/1.63G [01:14<00:00, 812kB/s]

[2025-08-19 12:13:34,989: INFO: 2249439211: Downloaded data from https://drive.google.com/file/d/1YYbpicJBzIF2QXHyPgQkoWrb1q_VQh6J/view?usp=sharing into file artifacts/data_ingestion/data.zip]
